In [1]:
import sys
sys.path.insert(0, "../")

from io import StringIO
import copy
import matplotlib.pyplot as plt
import neuron_morphology.swc_io as swcio
from neuron_morphology.morphology import Morphology
from neuron_morphology.swc_io import morphology_from_swc
from neuron_morphology.feature_extractor.data import Data
from neuron_morphology.feature_extractor.feature_extractor import FeatureExtractor
import neuron_morphology.feature_extractor.feature_writer as fw
from neuron_morphology.features.default_features import default_features

In [2]:
fmost_swc_file = "17545-6151-X24259-Y36270.swc"

In [3]:
swc_data = swcio.read_swc(fmost_swc_file)

In [4]:
nodes = swc_data.to_dict('record')

replace_type = 2 # default node type
for node in nodes:
    # unfortunately, pandas automatically promotes numeric types to float in to_dict
    node['parent'] = int(node['parent'])
    node['id'] = int(node['id'])
    node['type'] = int(node['type'])
    
    if node['parent'] == -1 and node['type'] != 1:
        replace_type = node['type']
        
    if node['type'] == 1 and node['parent'] != -1:
        node['type'] = replace_type
        
print(len(nodes), replace_type)

3397 2


In [5]:
soma_list = []
for node in nodes:
    if node['type'] == 1:
        soma_list.append(node)

print(soma_list)

# create a new soma point
if len(soma_list) > 1:
    x = 0
    y = 0
    z = 0
    n = len(soma_list)
    for node in soma_list:
        x += node['x']
        y += node['y']
        z += node['z']
        
    soma = copy.deepcopy(soma_list[0])
    soma['id'] = nodes[-1]['id']
    soma['x'] = x/n
    soma['y'] = y/n
    soma['z'] = z/n
    
    
    for node in soma_list:
        node['parent'] = soma['id']
        node['type'] = replace_type
else:
    soma = soma_list[0]

[{'id': 337101, 'type': 1, 'x': 7997.95, 'y': 3408.625, 'z': 3225.425, 'radius': 100.0, 'parent': -1}, {'id': 337205, 'type': 1, 'x': 7997.95, 'y': 3408.625, 'z': 3225.425, 'radius': 100.0, 'parent': -1}, {'id': 337210, 'type': 1, 'x': 7997.95, 'y': 3408.625, 'z': 3225.425, 'radius': 100.0, 'parent': -1}, {'id': 337367, 'type': 1, 'x': 7997.95, 'y': 3408.625, 'z': 3225.425, 'radius': 100.0, 'parent': -1}, {'id': 337383, 'type': 1, 'x': 7997.95, 'y': 3408.625, 'z': 3225.425, 'radius': 100.0, 'parent': -1}, {'id': 337389, 'type': 1, 'x': 7997.95, 'y': 3408.625, 'z': 3225.425, 'radius': 100.0, 'parent': -1}, {'id': 337400, 'type': 1, 'x': 7997.95, 'y': 3408.625, 'z': 3225.425, 'radius': 100.0, 'parent': -1}, {'id': 337433, 'type': 1, 'x': 7997.95, 'y': 3408.625, 'z': 3225.425, 'radius': 100.0, 'parent': -1}, {'id': 337457, 'type': 1, 'x': 7997.95, 'y': 3408.625, 'z': 3225.425, 'radius': 100.0, 'parent': -1}, {'id': 337464, 'type': 1, 'x': 7997.95, 'y': 3408.625, 'z': 3225.425, 'radius': 1

In [7]:
test_data = Data(Morphology(nodes, node_id_cb=lambda node: node['id'], parent_id_cb=lambda node: node['parent']))

In [8]:
fe = FeatureExtractor()
for feature in default_features:
    fe.register_features([feature])

In [9]:
feature_extraction_run = fe.extract(test_data)

../neuron_morphology/feature_extractor/mark.py:118: UserWarning: This morphology is not uniquely rooted! Found 278 root nodes. Features using the root node of this morphology may not select that node consistently. Some or all of these root nodes may not be soma nodes.
  f"This morphology is not uniquely rooted! Found {num_roots} "
2020-02-21 17:31:47,968 root         INFO     skipping mark (validation failed): type
2020-02-21 17:31:47,972 root         INFO     skipping mark (validation failed): type
2020-02-21 17:31:47,980 root         INFO     skipping mark (validation failed): type
2020-02-21 17:31:47,981 root         INFO     selected marks: <bound method FeatureExtractionRun.select_marks of <neuron_morphology.feature_extractor.feature_extraction_run.FeatureExtractionRun object at 0x7ffb5c722978>>
2020-02-21 17:31:47,982 root         INFO     skipping feature: apical_dendrite.bifurcation.dimension. Found extra marks: ['RequiresApical']
2020-02-21 17:31:47,984 root         INFO     s

2020-02-21 17:31:48,034 root         INFO     skipping feature: basal_dendrite.dendrite.earth_movers_distance. Found extra marks: ['RequiresLayeredPointDepths', 'RequiresReferenceLayerDepths']
2020-02-21 17:31:48,035 root         INFO     skipping feature: axon.dendrite.earth_movers_distance. Found extra marks: ['RequiresLayeredPointDepths', 'RequiresReferenceLayerDepths']
2020-02-21 17:31:48,037 root         INFO     selected features: ['basal_dendrite.bifurcation.dimension', 'all_neurites.bifurcation.dimension', 'axon.bifurcation.dimension', 'dendrite.bifurcation.dimension', 'basal_dendrite.node.dimension', 'all_neurites.node.dimension', 'axon.node.dimension', 'dendrite.node.dimension', 'basal_dendrite.compartment.dimension', 'all_neurites.compartment.dimension', 'axon.compartment.dimension', 'dendrite.compartment.dimension', 'basal_dendrite.tip.dimension', 'all_neurites.tip.dimension', 'axon.tip.dimension', 'dendrite.tip.dimension', 'basal_dendrite.num_nodes', 'all_neurites.num_node

In [10]:
heavy_path = "test_features.h5"
table_path = "test_features.csv"

features_writer = fw.FeatureWriter(heavy_path, table_path)
features_writer.add_run("test", feature_extraction_run.serialize())
features_writer.write_table()

../neuron_morphology/feature_extractor/feature_writer.py:54: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.heavy_file = h5py.File(self.heavy_path, driver="core")
2020-02-21 17:31:56,705 root         WARNING  writing additional outputs to csv. See output json for record of selected features and marks


In [11]:
print(feature_extraction_run.results)

{'basal_dendrite.bifurcation.dimension': {'height': nan, 'width': nan, 'depth': nan, 'min_xyz': array([nan, nan, nan]), 'max_xyz': array([nan, nan, nan]), 'bias_xyz': array([nan, nan, nan])}, 'all_neurites.bifurcation.dimension': {'width': 0.0, 'height': 0.0, 'depth': 0.0, 'min_xyz': array([ 1430.775, -2821.925,  -381.9  ]), 'max_xyz': array([ 1430.775, -2821.925,  -381.9  ]), 'bias_xyz': array([0., 0., 0.])}, 'axon.bifurcation.dimension': {'width': 0.0, 'height': 0.0, 'depth': 0.0, 'min_xyz': array([ 1430.775, -2821.925,  -381.9  ]), 'max_xyz': array([ 1430.775, -2821.925,  -381.9  ]), 'bias_xyz': array([0., 0., 0.])}, 'dendrite.bifurcation.dimension': {'height': nan, 'width': nan, 'depth': nan, 'min_xyz': array([nan, nan, nan]), 'max_xyz': array([nan, nan, nan]), 'bias_xyz': array([nan, nan, nan])}, 'basal_dendrite.node.dimension': {'width': 222.4250000000011, 'height': 201.60000000000127, 'depth': 196.77500000000055, 'min_xyz': array([ 967.7  , -284.375,  -72.85 ]), 'max_xyz': array